In [1]:
import pandas as pd
import warnings

from numpy.random import RandomState

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Open the file and study the general information

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')

print(data_0.info())
print(data_0.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647


In [3]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')

print(data_1.info())
print(data_1.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305


In [4]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

print(data_2.info())
print(data_2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


The info and head methods show that there aren't missing variables in any columns, and the data types are all correct. Our goal is to compare the data for the three different regions and select the region with the highest profit margin, analyzing the potential profit and risks using bootstrapping.

To do so, we'll first need to create models for each data set and compare them. Our target is product, while our features are f0, f1, and f2.

We need to do two things to prepare our data:
1. Remove columns that don't contribute to our model - in this case, it's the 'id' column
2. Split our data into training and validation sets with a 75:25 ratio
3. Standardize numerical columns

# Step 2: Prepare the data

Since the process of data preparation is the same for all models, let's first create a function for it.

In [5]:
warnings.filterwarnings("ignore") #filter warnings created by standardizing data on copied dataframe

#takes a data set and returns the training and validation sets for the data
def prep_data(data):
    
    #drop id
    data = data.drop(['id'], axis = 1)
    
    #get features and target
    features = data.drop(['product'], axis=1)
    target = data['product']
    
    #split data into training and validation sets
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    
    #standardize
    scaler = StandardScaler()
    numeric = ['f0','f1','f2']
    
    scaler.fit(features_train[numeric])
    
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    #return features and target for both training and validation sets
    return features_train, features_valid, target_train, target_valid

#call prep_data function for all data sets
d0_features_train, d0_features_valid, d0_target_train, d0_target_valid = prep_data(data_0)

d1_features_train, d1_features_valid, d1_target_train, d1_target_valid = prep_data(data_1)

d2_features_train, d2_features_valid, d2_target_train, d2_target_valid = prep_data(data_2)

#check
print(d0_features_train.head(5))

             f0        f1        f2
27212 -0.544828  1.390264 -0.094959
7866   1.455912 -0.480422  1.209567
62041  0.260460  0.825069 -0.204865
70185 -1.837105  0.010321 -0.147634
82230 -1.299243  0.987558  1.273181


Now we can create our models.

# Step 3: Train and test the model

We have to repeat the same steps again for all three data sets, so we'll create another method.

In [6]:
def get_prediction(features_train, features_valid, target_train):
    model = LinearRegression()
    
    model.fit(features_train, target_train)
    
    predicted_valid = model.predict(features_valid)
    
    return predicted_valid

d0_prediction = pd.Series(get_prediction(d0_features_train, d0_features_valid, d0_target_train))
d1_prediction = pd.Series(get_prediction(d1_features_train, d1_features_valid, d1_target_train))
d2_prediction = pd.Series(get_prediction(d2_features_train, d2_features_valid, d2_target_train))

print("Average volume of predicted reserves for data set 0:", d0_prediction.mean())
print("Average volume of predicted reserves for data set 1:", d1_prediction.mean())
print("Average volume of predicted reserves for data set 2:", d2_prediction.mean())

print("Data set 0 RMSE:", mean_squared_error(d0_target_valid, d0_prediction)**0.5)
print("Data set 1 RMSE:", mean_squared_error(d1_target_valid, d1_prediction)**0.5)
print("Data set 2 RMSE:", mean_squared_error(d2_target_valid, d2_prediction)**0.5)


Average volume of predicted reserves for data set 0: 92.59256778438038
Average volume of predicted reserves for data set 1: 68.728546895446
Average volume of predicted reserves for data set 2: 94.96504596800489
Data set 0 RMSE: 37.5794217150813
Data set 1 RMSE: 0.8930992867756158
Data set 2 RMSE: 40.02970873393434


If we go by average volume of predicted reserves, the obvious choice seems to be either the locations associated with data set 0 and 3. However, the root mean squared error for both of those sets is significantly higher than the RMSE for data set 1, meaning that the potential for a much lower profit than expected lies in those same locations. We'll have to further study the data to come to a better conclusion.

# Step 4: Prepare for profit calculation

"The budget for development of 200 oil wells is 100 USD million.

One barrel of raw materials brings 4.5 USD of revenue The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels)."

Considering these points, we'll **calculate the volume of reserves sufficient for developing a new well without losses**.

In [10]:
WELLS = 200
BUDGET= 100000000
USD_PER_BARREL = 4.5

budget_per_well = BUDGET/WELLS

products_ROI = (budget_per_well/USD_PER_BARREL) / 1000 #our target products is calculated by the thousand

print("Volume of reserves(by the thousand) sufficient for developing a new well without losses:",products_ROI)

Volume of reserves(by the thousand) sufficient for developing a new well without losses: 111.11111111111111


By finding the average budget required for a single well(budget/well), and dividing that by the revenue from one unit of product(4.5\*1000), we get the volume amount necessary for our ROI.

If we go by merely the average volume of predicted reserves for each region, none of the locations would be profitable, since we don't meet the minimum required reserves to break even. This means **we have to selectively choose locations in each region where the volume of predicted reserves is greater than the average for its region**.

Let's define a function to calculate profits, and utilize this function for each data set.

In [8]:
'''
our method takes 2 parameters - the target and predicted target
It will return the total profit for that particular set.
'''
def calculate_profit(target, predictions):
    #sort predicted target data
    
    sorted_probs = predictions.sort_values(ascending=False)
    #save wells with highest predicted values, and retrieve the true values for those wells
    selected = target[sorted_probs.index][:200]

    #calculate revenue
    revenue = usd_per_barrel*1000*selected.sum()
    
    #return total profit
    return round(revenue - budget, 2)

d0_target_valid = d0_target_valid.reset_index(drop=True)
d1_target_valid = d1_target_valid.reset_index(drop=True)
d2_target_valid = d2_target_valid.reset_index(drop=True)

print("Total profit of predicted top 200 wells of data set 0: ${} USD".format(calculate_profit(d0_target_valid, d0_prediction)))
print("Total profit of predicted top 200 wells of data set 1: ${} USD".format(calculate_profit(d1_target_valid, d1_prediction)))
print("Total profit of predicted top 200 wells of data set 2: ${} USD".format(calculate_profit(d2_target_valid, d2_prediction)))


Total profit of predicted top 200 wells of data set 0: $33208260.43 USD
Total profit of predicted top 200 wells of data set 1: $24150866.97 USD
Total profit of predicted top 200 wells of data set 2: $27103499.64 USD


Our calculations shows that the total profit of the top 200 predicted wells is highest for **data set 0**. If we were to base our decision from the top predictions, this would be our choice - however, realistically, our wells will not all be the most profitable.

# Step 5: Calculate risks and profit for each region

We'll use bootstrapping to determine the distribution of profits, and utilize the distribution of profits to determine the best region to build wells.

Let's define a method to use for the different data sets.

In [9]:
state = RandomState(12345)

def calculate_risks_and_profit(target, prediction):
    target = target.reset_index(drop=True)
    values = []
    negatives = 0
    for i in range(1000):
        predicted_subsample = prediction.sample(n=500, replace=True, random_state=state)
        target_subsample = target[predicted_subsample.index]
        profit = calculate_profit(target_subsample, predicted_subsample)
        values.append(profit)
        if profit < 0:
            negatives += 1
        
    values = pd.Series(values)
    
    mean = values.mean()

    lower = values.quantile(.025)
    upper = values.quantile(.975)
    
    risk_of_loss = negatives/1000
    
    print("Average profit: ${} USD".format(mean))
    print("95% confidence interval: ({},{})".format(lower, upper))
    print("Risk of loss: {0:.0%}".format(risk_of_loss))

print("Region 0:")
calculate_risks_and_profit(d0_target_valid, d0_prediction)

print("\nRegion 1")
calculate_risks_and_profit(d1_target_valid, d1_prediction)

print("\nRegion 2")
calculate_risks_and_profit(d2_target_valid, d2_prediction)

Region 0:
Average profit: $4259385.26927 USD
95% confidence interval: (-1020900.94925,9479763.53575)
Risk of loss: 6%

Region 1
Average profit: $5182594.936989999 USD
95% confidence interval: (1281232.31,9536129.818999998)
Risk of loss: 0%

Region 2
Average profit: $4201940.0534500005 USD
95% confidence interval: (-1158526.08925,9896299.40225)
Risk of loss: 6%


Based on our studies, **Region 1** should be our choice for developing new wells. The variance of average profit in region 0 and region 2 is much greater than region 1, and the risk of loss is significant, at a 6% probability of not making a profit. Additionally, the highest average profit lies in region 1, making it a safe choice to develop.